## 1. ResNet

- ResNet은 CNN 중에서 가장 많이 쓰이는 모델
- VGG는 19층 이상으로 쌓을 수 없다. 다만 ResNet은 Skip Connection 기술을 이용해서 기울시 소실을 방지함

In [1]:
import torch
import torch.nn as nn

## 2. ResNet 기본 블록 정의하기

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(BasicBlock, self).__init__()
        
        self.c1 = nn.Conv2d(in_channels, out_channels, 
                            kernel_size=kernel_size, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels, 
                            kernel_size=kernel_size, padding=1)
        self.downsample = nn.Conv2d(in_channels, out_channels, 
                                    kernel_size=1)
        
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x_ = x
        
        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)
        
        # 합성곱의 결과와 입력의 채널 수를 맞춤
        x_ = self.downsample(x_)
        
        x += x_
        
        x = self.relu(x)
        
        return x 
        

## 3. ResNet 모델 정의하기

In [3]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)
        
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.pool(x)
        x = self.b3(x)
        x = self.pool(x)
        
        x = torch.flatten(x, start_dim=1)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
                
        return x

## 4. 데이터 전처리 정의

In [4]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import RandomCrop, RandomHorizontalFlip
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

transforms = Compose([
    RandomCrop((32,32), padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

## 5. 데이터 불러오기

In [5]:
training_data = CIFAR10(root='./', train=True, download=True, transform=transforms)
test_data = CIFAR10(root='./', train=False, download=True, transform=transforms)

train_loader = DataLoader(dataset=training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## 6. 모델 정의하기

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet(num_classes=10)
model.to(device)

ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


## 7. 학습 루프 정의하기

In [8]:
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)
max_acc = 0

for epoch in range(30):
    iterator = tqdm.tqdm(train_loader)
    
    for data, label in iterator:
        optim.zero_grad()
        
        preds = model(data.to(device))
        
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch: {epoch+1} loss:{loss.item()}")
    
    with torch.no_grad():
        iterator = tqdm.tqdm(test_loader)

        acc = 0 
        
        for data, label in iterator:
            preds = model(data.to(device))
            
            output = preds.data.max(1)[1]
            corr = output.eq(label.to(device).data).sum().item()
            acc += corr
            
        
        acc = acc / len(test_data)
        print(f"Accuracy: {acc:.4f}")
        
        if max_acc < acc:
            max_acc = acc
            print("Save Pth!")
            torch.save(model.state_dict(), "ResNet.pth")
        

100%|██████████| 157/157 [00:01<00:00, 121.36it/s]


Accuracy: 0.6916
Save Pth!


100%|██████████| 157/157 [00:01<00:00, 120.90it/s]


Accuracy: 0.7324
Save Pth!


100%|██████████| 157/157 [00:01<00:00, 120.80it/s]


...

Accuracy: 0.8726


100%|██████████| 157/157 [00:01<00:00, 121.71it/s]


Accuracy: 0.8823
Save Pth!


100%|██████████| 157/157 [00:01<00:00, 122.03it/s]

Accuracy: 0.8820
